# Part 4: XML creation
## Dependencies: OTIO and pandas



In [1]:
#audio
from __future__ import print_function
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import librosa
import librosa.display

In [2]:
import pandas as pd 
import pyloudnorm as pyln

In [3]:
#video
import moviepy
from moviepy.editor import *
import opentimelineio as otio
import argparse

In [4]:
import cv2

### Quick summary, OTIO can output XML that CAN be used to import timelines. We just need to learn the behavior and then create a loop to automate clipping, using a imported dataframe as a outline.

# Timeline Outline behavior testing
- First create a list of videos
    - sfs
- Sec create list of audio

- Link audio to video, IE if mic1 use center camera

## Use the lists of camera and audio to createa loops creating timeranges

In [5]:
AUDIO_VIDEO_TUPLE_LIST=[("E2 - Chukwu.mp3", 1), ("E2 - Osi.mp3", 1),("E2 - Crystal.mp3",2),("E2 - Scott.mp3",2)]

VIDEO_FILEPATH_LIST= ["Hang out 2 Middle.mp4","Hang out 2 Left.mp4","Hang out 2 Right.mp4"]

# Further data editing 

## Create a camera view column that matches the audio_video_tuple to  idxmax

In [6]:
data_df=pd.read_pickle('idxmax.pkl')

In [7]:
#use indexes of when crossover is 1 to change cam_view to 0 for three seconds after crossover

In [8]:

cap = cv2. VideoCapture(VIDEO_FILEPATH_LIST[0])
vlength = int(cap. get(cv2. CAP_PROP_FRAME_COUNT))
alength=len(data_df)

In [9]:
alength
vlength

103904

In [10]:
vrate=24
arate=500
arate_actual=24

## Create intervals of data in tuple form. 

## idxmax_mic_data are Video Tuples which  represent the start and end frame of main speaker. Can be used for audio data as well when there is sufficient mic bleed.

## threshold_mic_data are Audio Tuples represent the start and end frame of when mic is being used.


## Main cam tuples use the main cam column in the dataframe to change to center cam

In [11]:
def dataframe_getintervals(series,desiredvalue):
    #make sure series is the df['column']
    t=series.index[series==desiredvalue].to_series()
    interval_list=t.groupby(t.diff().ne(1).cumsum()).agg(['first','last']).apply(tuple,1).tolist()
    
    return interval_list

In [12]:
#create lists of tuples for each camera based on the audio

list_of_idxmax_mic_data=[]
for i in range(len(AUDIO_VIDEO_TUPLE_LIST)+1):
    tuple_list=dataframe_getintervals(data_df['idxmax'],i)
    list_of_idxmax_mic_data.append(tuple_list)

    
##create lists of tuples for each Audio

In [13]:
#Audio Tuples
#creating a list to referencing the column names of df 
list_str_audio_thresholds=[]
for irow in range(len(AUDIO_VIDEO_TUPLE_LIST)):
    varname="A"
    list_str_audio_thresholds.append(varname+str(irow))

In [14]:
#create lists of tuples for each audio based the threshold region
list_of_tuples_threshold_mic_data=[]
for i in range(len(AUDIO_VIDEO_TUPLE_LIST)):
    tuple_list=dataframe_getintervals(data_df[list_str_audio_thresholds[i]],1)
    list_of_tuples_threshold_mic_data.append(tuple_list)

In [15]:
# main cam tuple list
main_cam_tuple_list=dataframe_getintervals(data_df['Main_cam'],1)

In [16]:
len(main_cam_tuple_list)

174

In [17]:
main_cam_tuple_list_corrected=[]
guard=0
count=0
for i in range(len(main_cam_tuple_list)):
    
    if guard==0:
        
        if i+1==len(main_cam_tuple_list):
            pass
        else:
            if main_cam_tuple_list[i+1][0]-main_cam_tuple_list[i][1]<arate*3:
                #create tuple
                new_tuple=(main_cam_tuple_list[i][0],main_cam_tuple_list[i+1][1])
                #add tuple to list
                main_cam_tuple_list_corrected.append(new_tuple)
                guard=1

            else:
                main_cam_tuple_list_corrected.append(main_cam_tuple_list[i])
    
    else:
        guard=0

In [18]:
extended_audio_idxmax_mic_data=[]

for i in range(len(list_of_idxmax_mic_data)):
#for each list of tuple in the list
    new_tuple_list=[]
    
    for i2 in range(len(list_of_idxmax_mic_data[i])):
    #for each element in tuple, add 1.5*rate to 
        
        extended_tuple=(list_of_idxmax_mic_data[i][i2][0]+arate*1.5,list_of_idxmax_mic_data[i][i2][1]+arate*1.5)
        
        
        
        new_tuple_list.append(extended_tuple)
        
 
    
    extended_audio_idxmax_mic_data.append(new_tuple_list)


extended_audio_idxmax_mic_data_corrected=
guard=0
count=0
    if guard==0:

        if i+1==len(list_of_idxmax_mic_data):
            pass
        else:
            if main_cam_tuple_list[i+1][0]-main_cam_tuple_list[i][1]<arate*3:
                #create tuple
                new_tuple=(main_cam_tuple_list[i][0],main_cam_tuple_list[i+1][1])
                #add tuple to list
                main_cam_tuple_list_corrected.append(new_tuple)
                guard=1

            else:
                main_cam_tuple_list_corrected.append(main_cam_tuple_list[i])

    else:
                guard=0

In [19]:
#skeletal outline of otio


# build the structure
tl = otio.schema.Timeline(name="Example timeline")

# add track for each video file and each audio file
#for each file add a track

#create lists for each track to reference back to later
#vtr is video track, etc.
vtr_list=[]
atr_list=[]


#add a audio AND video track for each video track
    #default cam first because it is lowest priority
vtr_default = otio.schema.Track(name="Default_camera", kind="Video")
tl.tracks.append(vtr_default)

for i in AUDIO_VIDEO_TUPLE_LIST:
    atr = otio.schema.Track(name=i[0], kind="Audio")
    tl.tracks.append(atr)
    atr_list.append(atr)
    
    #video
    vtr = otio.schema.Track(name=i[0]+"_video", kind="Video")
    tl.tracks.append(vtr)
    vtr_list.append(vtr)

# add main cam
main_tr = otio.schema.Track(name='Main_Cam')
tl.tracks.append(main_tr)


In [20]:
vrate=24
arate=500
arate_actual=24

# Two different loops, One for Video, another for audio

In [21]:
#video clips
# i is to keep track which audio file we are currently on
i=-1

# i2 is to keep track of how many clips there are
i2=0

for lists_of_tuples in list_of_idxmax_mic_data:
    #connect the list of tuples to the audio file
    #ignore 0 for now. We will come back to this later, i will still be -1 but we will add a default clip instead
    
    if i==-1:
    #add the entire default cam clip in the lowest priority
        #vclip
        #aclip
        pass
    
    else: 
        #print(AUDIO_VIDEO_TUPLE_LIST[i],VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]])
    
        # Connect the audio and video tracks 
        atrack=atr_list[i]
        afname=AUDIO_VIDEO_TUPLE_LIST[i][0]
        
        # AUDIO_VIDEO_TUPLE_LIST[i][1] references the VIDEO_FILEPATH_LIST to determine which video file to use
        vtrack=vtr_list[i]
        vfname=VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]]
        
        
        
        #will remember the previous end of clip's timecode
        #will reset to 0 when a video-audio track pair is done
        vprevious_end_timecode=0
        aprevious_end_timecode=0
        
    #=========================================================================
           
        for tuples in lists_of_tuples:
            
        #=====================================================
          #adding variables for time and duration calculation
            #Video clips 
            v_clip_starttime=tuples[0]/arate*vrate
            v_clip_duration=tuples[1]/arate*vrate-v_clip_starttime

            
        #Specifying the start time and end time of the video file where clips come from
            v_clip_available_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, vrate),
            duration=otio.opentime.RationalTime(vlength, vrate))
            
            vref = otio.schema.ExternalReference(target_url=vfname,
            available_range=v_clip_available_range)
            
            
        #specifying where the start timecode and end timecode of the clip is
            v_clip_source_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(v_clip_starttime, vrate),
                duration=otio.opentime.RationalTime(v_clip_duration, vrate))



        #Video Gaps
            #the gap length is equal to (timecode of next clip time - timecode end of previous clip)
            v_gap_start_time=0
            v_gap_duration=v_clip_starttime-vprevious_end_timecode
            
            v_gap_timerange=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(v_gap_start_time, vrate),
                duration=otio.opentime.RationalTime(v_gap_duration, vrate))
            
                
            
            
         #=====================================================   
            # adding gaps before adding clips
            #convert the tuple ranges from audio rate to video rate
                #audio rate=500
                #video rate=24



            # create gap settings
            vgap = otio.schema.Gap(
                name="vGap{}".format(i2 + 1),

                # available_range_from_list is the 
                source_range=otio.opentime.TimeRange(
                    start_time=otio.opentime.RationalTime(
                        v_gap_timerange.start_time.value,
                        v_gap_timerange.start_time.rate
                    ),
                    duration=otio.opentime.RationalTime(
                        v_gap_timerange.duration.value,
                        v_gap_timerange.duration.rate
                    ),
                )
            )

            # put the clip into the track
            vtrack.append(vgap)



        #=======================================================
        # adding Video clips 
        #convert the tuple ranges from audio rate to video rate
        #audio rate=500

                        #add clip to track                

            vcl = otio.schema.Clip(
                        name="vClip{}".format(i2 + 1),
                        media_reference=vref,

                        # available_range_from_list is the 
                        source_range=otio.opentime.TimeRange(
                            start_time=otio.opentime.RationalTime(
                                v_clip_source_range.start_time.value,
                                v_clip_source_range.start_time.rate
                            ),
                            duration=otio.opentime.RationalTime(
                                v_clip_source_range.duration.value,
                                v_clip_source_range.duration.rate
                            ),
                        )
                    )

            vtrack.append(vcl)

            vprevious_end_timecode=tuples[1]/arate*vrate
            #apreviousduration=
        


            i2=i2+1

            
    i=i+1
    #if i == 2:
        #break

# Main speaker audio method

In [22]:
"""# i is to keep track which audio file we are currently on
i=-1

# i2 is to keep track of how many clips there are
i2=0

for lists_of_tuples in list_of_idxmax_mic_data:
    #connect the list of tuples to the audio file
    #ignore 0 for now. We will come back to this later, i will still be -1 but we will add a default clip instead
    
    if i==-1:
    #add the entire default cam clip in the lowest priority
        #vclip
        #aclip
        pass
    
    else: 
        print(AUDIO_VIDEO_TUPLE_LIST[i],VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]])
    
        # Connect the audio and video tracks 
        atrack=atr_list[i]
        afname=AUDIO_VIDEO_TUPLE_LIST[i][0]
        
        # AUDIO_VIDEO_TUPLE_LIST[i][1] references the VIDEO_FILEPATH_LIST to determine which video file to use
        vtrack=vtr_list[i]
        vfname=VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]]
        
        
        
        #will remember the previous end of clip's timecode
        #will reset to 0 when a video-audio track pair is done
        vprevious_end_timecode=0
        aprevious_end_timecode=0
        
    #=========================================================================
           
        for tuples in lists_of_tuples:
            
        #=====================================================
          #adding variables for time and duration calculation
            #Video clips 
            a_clip_starttime=tuples[0]/arate*arate_actual
            a_clip_duration=tuples[1]/arate*arate_actual-a_clip_starttime

            
        #Specifying the start time and end time of the video file where clips come from
            a_clip_available_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, arate_actual),
            duration=otio.opentime.RationalTime(alength*arate_actual, arate_actual))
            
            aref = otio.schema.ExternalReference(target_url=afname,
            available_range=a_clip_available_range)
            
            
        #specifying where the start timecode and end timecode of the clip is
            a_clip_source_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(a_clip_starttime, arate_actual),
                duration=otio.opentime.RationalTime(a_clip_duration, arate_actual))



        #Video Gaps
            #the gap length is equal to (timecode of next clip time - timecode end of previous clip)
            a_gap_start_time=0
            a_gap_duration=a_clip_starttime-aprevious_end_timecode
            
            a_gap_timerange=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(a_gap_start_time, arate_actual),
                duration=otio.opentime.RationalTime(a_gap_duration, arate_actual))
            
                
            
            
         #=====================================================   
            # adding gaps before adding clips
            #convert the tuple ranges from audio rate to video rate
                #audio rate=500
                #video rate=24



            # create gap settings
            agap = otio.schema.Gap(
                name="vGap{}".format(i2 + 1),

                # available_range_from_list is the 
                source_range=otio.opentime.TimeRange(
                    start_time=otio.opentime.RationalTime(
                        a_gap_timerange.start_time.value,
                        a_gap_timerange.start_time.rate
                    ),
                    duration=otio.opentime.RationalTime(
                        a_gap_timerange.duration.value,
                        a_gap_timerange.duration.rate
                    ),
                )
            )

            # put the clip into the track
            atrack.append(agap)



        #=======================================================
        # adding Video clips 
        #convert the tuple ranges from audio rate to video rate
        #audio rate=500

                        #add clip to track                

            acl = otio.schema.Clip(
                        name="aClip{}".format(i2 + 1),
                        media_reference=aref,

                        # available_range_from_list is the 
                        source_range=otio.opentime.TimeRange(
                            start_time=otio.opentime.RationalTime(
                                a_clip_source_range.start_time.value,
                                a_clip_source_range.start_time.rate
                            ),
                            duration=otio.opentime.RationalTime(
                                a_clip_source_range.duration.value,
                                a_clip_source_range.duration.rate
                            ),
                        )
                    )

            atrack.append(acl)

            aprevious_end_timecode=tuples[1]/arate*arate_actual
            #apreviousduration=
        


            i2=i2+1

            
    i=i+1
    #if i == 2:
        #break"""

'# i is to keep track which audio file we are currently on\ni=-1\n\n# i2 is to keep track of how many clips there are\ni2=0\n\nfor lists_of_tuples in list_of_idxmax_mic_data:\n    #connect the list of tuples to the audio file\n    #ignore 0 for now. We will come back to this later, i will still be -1 but we will add a default clip instead\n    \n    if i==-1:\n    #add the entire default cam clip in the lowest priority\n        #vclip\n        #aclip\n        pass\n    \n    else: \n        print(AUDIO_VIDEO_TUPLE_LIST[i],VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]])\n    \n        # Connect the audio and video tracks \n        atrack=atr_list[i]\n        afname=AUDIO_VIDEO_TUPLE_LIST[i][0]\n        \n        # AUDIO_VIDEO_TUPLE_LIST[i][1] references the VIDEO_FILEPATH_LIST to determine which video file to use\n        vtrack=vtr_list[i]\n        vfname=VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]]\n        \n        \n        \n        #will remember the previous end of clip\

# Loudness Threshold Audio method

In [23]:
# i is to keep track which audio file we are currently on
i=0

# i2 is to keep track of how many clips there are
i2=0

for lists_of_tuples in list_of_tuples_threshold_mic_data:
    #connect the list of tuples to the audio file
    #ignore 0 for now. We will come back to this later, i will still be -1 but we will add a default clip instead
    
    if i==-1:
    #add the entire default cam clip in the lowest priority
        #vclip
        #aclip
        pass
    
    else: 
        print(AUDIO_VIDEO_TUPLE_LIST[i],VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]])
    
        # Connect the audio and video tracks 
        atrack=atr_list[i]
        afname=AUDIO_VIDEO_TUPLE_LIST[i][0]
        
        # AUDIO_VIDEO_TUPLE_LIST[i][1] references the VIDEO_FILEPATH_LIST to determine which video file to use
        vtrack=vtr_list[i]
        vfname=VIDEO_FILEPATH_LIST[AUDIO_VIDEO_TUPLE_LIST[i][1]]
        
        
        
        #will remember the previous end of clip's timecode
        #will reset to 0 when a video-audio track pair is done
        vprevious_end_timecode=0
        aprevious_end_timecode=0
        
    #=========================================================================
           
        for tuples in lists_of_tuples:
            
        #=====================================================
          #adding variables for time and duration calculation
            #Video clips 
            a_clip_starttime=tuples[0]/arate*arate_actual
            a_clip_duration=tuples[1]/arate*arate_actual-a_clip_starttime

            
        #Specifying the start time and end time of the video file where clips come from
            a_clip_available_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(0, arate_actual),
            duration=otio.opentime.RationalTime(alength*arate_actual, arate_actual))
            
            aref = otio.schema.ExternalReference(target_url=afname,
            available_range=a_clip_available_range)
            
            
        #specifying where the start timecode and end timecode of the clip is
            a_clip_source_range=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(a_clip_starttime, arate_actual),
                duration=otio.opentime.RationalTime(a_clip_duration, arate_actual))



        #Video Gaps
            #the gap length is equal to (timecode of next clip time - timecode end of previous clip)
            a_gap_start_time=0
            a_gap_duration=a_clip_starttime-aprevious_end_timecode
            
            a_gap_timerange=otio.opentime.TimeRange(
                start_time=otio.opentime.RationalTime(a_gap_start_time, arate_actual),
                duration=otio.opentime.RationalTime(a_gap_duration, arate_actual))
            
                
            
            
         #=====================================================   
            # adding gaps before adding clips
            #convert the tuple ranges from audio rate to video rate
                #audio rate=500
                #video rate=24



            # create gap settings
            agap = otio.schema.Gap(
                name="vGap{}".format(i2 + 1),

                # available_range_from_list is the 
                source_range=otio.opentime.TimeRange(
                    start_time=otio.opentime.RationalTime(
                        a_gap_timerange.start_time.value,
                        a_gap_timerange.start_time.rate
                    ),
                    duration=otio.opentime.RationalTime(
                        a_gap_timerange.duration.value,
                        a_gap_timerange.duration.rate
                    ),
                )
            )

            # put the clip into the track
            atrack.append(agap)



        #=======================================================
        # adding Video clips 
        #convert the tuple ranges from audio rate to video rate
        #audio rate=500

                        #add clip to track                

            acl = otio.schema.Clip(
                        name="aClip{}".format(i2 + 1),
                        media_reference=aref,

                        # available_range_from_list is the 
                        source_range=otio.opentime.TimeRange(
                            start_time=otio.opentime.RationalTime(
                                a_clip_source_range.start_time.value,
                                a_clip_source_range.start_time.rate
                            ),
                            duration=otio.opentime.RationalTime(
                                a_clip_source_range.duration.value,
                                a_clip_source_range.duration.rate
                            ),
                        )
                    )

            atrack.append(acl)

            aprevious_end_timecode=tuples[1]/arate*arate_actual
            #apreviousduration=
        


            i2=i2+1

            
    i=i+1
    #if i == 2:
        #break

('E2 - Chukwu.mp3', 1) Hang out 2 Left.mp4
('E2 - Osi.mp3', 1) Hang out 2 Left.mp4
('E2 - Crystal.mp3', 2) Hang out 2 Right.mp4
('E2 - Scott.mp3', 2) Hang out 2 Right.mp4


# Main cam column


In [24]:
vtrack=main_tr
vfname=VIDEO_FILEPATH_LIST[0]

#will remember the previous end of clip's timecode
#will reset to 0 when a video-audio track pair is done
vprevious_end_timecode=0
aprevious_end_timecode=0

#=========================================================================
i=0
i2=0
for tuples in main_cam_tuple_list_corrected:

#=====================================================
  #adding variables for time and duration calculation
    #Video clips 
    v_clip_starttime=tuples[0]/arate*vrate
    v_clip_duration=tuples[1]/arate*vrate-v_clip_starttime


#Specifying the start time and end time of the video file where clips come from
    v_clip_available_range=otio.opentime.TimeRange(
    start_time=otio.opentime.RationalTime(0, vrate),
    duration=otio.opentime.RationalTime(vlength, vrate))

    vref = otio.schema.ExternalReference(target_url=vfname,
    available_range=v_clip_available_range)


#specifying where the start timecode and end timecode of the clip is
    v_clip_source_range=otio.opentime.TimeRange(
        start_time=otio.opentime.RationalTime(v_clip_starttime, vrate),
        duration=otio.opentime.RationalTime(v_clip_duration, vrate))



#Video Gaps
    #the gap length is equal to (timecode of next clip time - timecode end of previous clip)
    v_gap_start_time=0
    v_gap_duration=v_clip_starttime-vprevious_end_timecode

    v_gap_timerange=otio.opentime.TimeRange(
        start_time=otio.opentime.RationalTime(v_gap_start_time, vrate),
        duration=otio.opentime.RationalTime(v_gap_duration, vrate))




 #=====================================================   
    # adding gaps before adding clips
    #convert the tuple ranges from audio rate to video rate
        #audio rate=500
        #video rate=24



    # create gap settings
    vgap = otio.schema.Gap(
        name="vGap{}".format(i2 + 1),

        # available_range_from_list is the 
        source_range=otio.opentime.TimeRange(
            start_time=otio.opentime.RationalTime(
                v_gap_timerange.start_time.value,
                v_gap_timerange.start_time.rate
            ),
            duration=otio.opentime.RationalTime(
                v_gap_timerange.duration.value,
                v_gap_timerange.duration.rate
            ),
        )
    )

    # put the clip into the track
    vtrack.append(vgap)



#=======================================================
# adding Video clips 
#convert the tuple ranges from audio rate to video rate
#audio rate=500

                #add clip to track                

    vcl = otio.schema.Clip(
                name="vClip{}".format(i2 + 1),
                media_reference=vref,

                # available_range_from_list is the 
                source_range=otio.opentime.TimeRange(
                    start_time=otio.opentime.RationalTime(
                        v_clip_source_range.start_time.value,
                        v_clip_source_range.start_time.rate
                    ),
                    duration=otio.opentime.RationalTime(
                        v_clip_source_range.duration.value,
                        v_clip_source_range.duration.rate
                    ),
                )
            )

    vtrack.append(vcl)

    vprevious_end_timecode=tuples[1]/arate*vrate
    #apreviousduration=



    i2=i2+1


i=i+1
#if i == 2:
#break

In [25]:
otio.adapters.write_to_file(tl, 'hangout2_threshold.kdenlive')
otio.adapters.write_to_file(tl, 'hangout2_threshold.xml')

'hangout2_threshold.xml'

In [26]:
#otio.adapters.write_to_file(tl, 'videoaudio_beta2_main.kdenlive')
#otio.adapters.write_to_file(tl, 'videoaudio_beta2_main.xml')

# The clip adding via tuple looping